## Ohmic bath
We first create Ohmic bath with the following parameters

In [ ]:
using QuantumAnnealingTools, OrdinaryDiffEq, Plots, Printf, LaTeXStrings

β = 4
T = β_2_temperature(β)
η = 0.1
fc= 10/(2π)
bath = Ohmic(η, fc, T)

The spectrum $\gamma$ is plotted below:

In [ ]:
plot(bath, :γ, range(0,10,length=100), linewidth=2, label="")

and the properties of the bath are

In [ ]:
τsb, err_τsb = τ_SB((x)->correlation(x, bath))
@printf("τ_sb of the Ohmic bath is %.6f with error estimation %.2e \n", τsb, err_τsb)
τb, err_τb = τ_B((x)->correlation(x, bath), 100, τsb)
@printf("τ_b of the Ohmic bath is %.6f with error estimation %.2e \n", τb, err_τb)

## Annealing

We define the annealing process as

In [ ]:
Hp = 0.5*σz⊗σi - 0.7*σi⊗σz + 0.3*σz⊗σz
Hd = standard_driver(2)
H = DenseHamiltonian([(s)->1-s, (s)->s], [-Hd, Hp], unit=:ħ)

The spectrum of the Hamiltonian during the evolution is

In [ ]:
plot(H, range(0,1,length=100), 4, linewidth=2)
xlabel!("s")
ylabel!(L"P(s)")

### Closed system

We now run closed-system simulation

In [ ]:
tf = 20
ρ0 = (σi+σx)⊗(σi+σx)/4
coupling = ConstantCouplings([σz⊗σi, σi⊗σz], unit=:ħ)
annealing = Annealing(H, ρ0, bath=bath, coupling=coupling)
close_sol = solve_von_neumann(annealing, tf, alg = Tsit5(), abstol=1e-6, reltol=1e-6);

The population of instantanous ground state is:

In [ ]:
plot(close_sol, H, 1, range(0,tf,length=100), linewidth=2)
xlabel!("t")
ylabel!(L"P_G(s)")

The populations of the computational states are:

In [ ]:
t_axis = range(0,tf,length=100)
p_computational_basis = [real(diag(close_sol(s))) for s in t_axis]
p_computational_basis = hcat(p_computational_basis...)
plot(t_axis, p_computational_basis', linewidth=2, label=[L"\rho_{00}" L"\rho_{11}" L"\rho_{22}" L"\rho_{33}"])
xlabel!("t")
ylabel!(L"\rho")

### Redfield equation

We solve the Redfield equation:

In [ ]:
tf = 20
U = solve_unitary(annealing, tf, alg = Tsit5(), abstol=1e-7, reltol=1e-7);
redfield_sol = solve_redfield(annealing, tf, U, alg = Tsit5(), abstol=1e-7, reltol=1e-7);

We plot the populations of the computational states:

In [ ]:
t_axis = range(0,tf,length=100)
p_computational_basis = [real(diag(redfield_sol(s))) for s in t_axis]
p_computational_basis = hcat(p_computational_basis...)
plot(t_axis, p_computational_basis', linewidth=2, label=[L"\rho_{00}" L"\rho_{11}" L"\rho_{22}" L"\rho_{33}"])
xlabel!("t")
ylabel!(L"\rho")

We can see that the density matrix becomes negative during the evolution.

### Positivity check

We can add a callback to stop the ODE solver when the density matrix become negative.

In [ ]:
redfield_sol = solve_redfield(annealing, tf, U, alg = Tsit5(), abstol=1e-7, reltol=1e-7, callback=PositivityCheckCallback())